In [1]:
from cobra.io import read_sbml_model
model=read_sbml_model('Models/yeast8.xml')

In [2]:
for m in model.medium.items():
    print(model.reactions.get_by_id(m[0]), "|", model.reactions.get_by_id(m[0]).name, "| conc. =", m[1])

r_1654: s_0420 <=>  | ammonium exchange | conc. = 1000.0
r_1714: s_0565 <=>  | D-glucose exchange | conc. = 1.0
r_1832: s_0796 <=>  | H+ exchange | conc. = 1000.0
r_1861: s_0925 <=>  | iron(2+) exchange | conc. = 1000.0
r_1992: s_1277 <=>  | oxygen exchange | conc. = 1000.0
r_2005: s_1324 <=>  | phosphate exchange | conc. = 1000.0
r_2020: s_1374 <=>  | potassium exchange | conc. = 1000.0
r_2049: s_1438 <=>  | sodium exchange | conc. = 1000.0
r_2060: s_1468 <=>  | sulphate exchange | conc. = 1000.0
r_2100: s_0805 <=>  | water exchange | conc. = 1000.0
r_4593: s_4200 <=>  | chloride exchange | conc. = 1000.0
r_4594: s_4201 <=>  | Cu2(+) exchange | conc. = 1000.0
r_4595: s_4202 <=>  | Mn(2+) exchange | conc. = 1000.0
r_4596: s_4203 <=>  | Zn(2+) exchange | conc. = 1000.0
r_4597: s_4204 <=>  | Mg(2+) exchange | conc. = 1000.0
r_4600: s_4199 <=>  | Ca(2+) exchange | conc. = 1000.0


In [3]:
# Maximum growth and progesterone productivity/yield:
with model:
    baseline_max_growth = model.optimize().objective_value
    model.objective = model.reactions.r_1093
    baseline_max_production = model.optimize().objective_value
    glucose_flux = model.reactions.r_1714.flux
    baseline_max_yield = baseline_max_production/(-1 * glucose_flux)
print(f"Maximum growth: {round(baseline_max_growth,3)} /h")
print(f"Maximum progesterone productivity: {round(baseline_max_production,3)} mmol/gDW*h")
print(f"Maximum progesterone yield: {round(baseline_max_yield,3)} mmol progesterone/mmol glucose")

Maximum growth: 0.082 /h
Maximum progesterone productivity: 0.0 mmol/gDW*h
Maximum progesterone yield: 0.0 mmol progesterone/mmol glucose


In [4]:
# Production for a dual model objective of BIOMASS and progesterone:
with model: 
    medium = model.medium
    model.medium=medium
    Glucose_exchange = model.reactions.get_by_id('r_1714') 

    model.objective = {model.reactions.get_by_id('r_2111'): baseline_max_production, 
                    model.reactions.get_by_id('r_1093'): baseline_max_growth} # objective is to maximise the production of progesterone while also growing as much as possible
    solution = model.optimize()
    xylitol_production = solution.fluxes['r_1093']
    glucose_flux = model.reactions.r_1718.flux
    max_yield = xylitol_production/(-1 * glucose_flux)
    print('Glucose uptake:',format(medium[Glucose_exchange.id]), 'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['r_2111'],3), '/h')
    print('Maximum xylitol productivity:', round(xylitol_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum xyilitol yield:', round(max_yield,3), 'mmol xylitol/mmol glucose') # calculate yield on galactose

Glucose uptake: 1.0 mmol/gDW*h
Maximum growth: 0.082 /h
Maximum xylitol productivity: 0.0 mmol/gDW*h
Maximum xyilitol yield: nan mmol xylitol/mmol glucose


C:\Users\pauli\AppData\Local\Temp\ipykernel_41052\3379447133.py:12: RuntimeWarning: invalid value encountered in scalar divide
  max_yield = xylitol_production/(-1 * glucose_flux)


In [5]:
# Growth and progesterone yield/productivity on galactose:
with model: #galactose
    Glucose_exchange = model.reactions.get_by_id('r_1714') #define glucose reaction
    Galactose_exchange = model.reactions.get_by_id('r_1718') #define galactose reaction
    print('Medium with glucose (before):', model.medium)
    print('')
    medium=model.medium
    medium[Glucose_exchange.id] = 0 # Set glucose to zero
    medium[Galactose_exchange.id] = 1.0 # add galactose to medium
    model.medium = medium
    print('Medium with galactose (after):',model.medium)
    print() 
    print('When objective is set to only progesterone production:')
    model.objective = model.reactions.r_1093 # Objective is to lead flux through the reaction producing progesterone to maximise progesterone production 
    solution1 = model.optimize()
    solution = model.optimize().objective_value
    progesterone_production = solution
    galactose_flux = model.reactions.r_1718.flux
    max_yield_gal = progesterone_production/(-1 * galactose_flux)
    print('Galactose uptake:',format(medium[Galactose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution1.fluxes['r_2111'],3), '/h')
    print('Maximum progesterone productivity on galactose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on galactose:', round(max_yield_gal,3), 'mmol progesterone/mmol galactose') # calculate yield on galactose

with model: #galactose
    print()
    print('When objective is set at both growth and progesterone production:')
    Glucose_exchange = model.reactions.get_by_id('r_1714') #define glucose reaction
    Galactose_exchange = model.reactions.get_by_id('r_1718') #define galactose reaction
    medium = model.medium
    print('Medium before:', model.medium)
    print('')
    medium[Glucose_exchange.id] = 0 # Set glucose to zero
    medium[Galactose_exchange.id] = 1.0 # add galactose to medium
    model.medium = medium
    #print('Medium after:',model.medium)
    #print('') 
    model.objective = {model.reactions.get_by_id('r_2111'): baseline_max_production, 
                    model.reactions.get_by_id('r_1093'): baseline_max_growth} # objective is to maximise the production of progesterone while also growing as much as possible
    solution = model.optimize()
    progesterone_production = solution.fluxes['r_1093']
    galactose_flux = model.reactions.r_1718.flux
    max_yield_gal = progesterone_production/(-1 * galactose_flux)
    print('Galactose uptake:',format(medium[Galactose_exchange.id]),'mmol/gDW*h')
    print('Maximum growth:', round(solution.fluxes['r_2111'],3), '/h')
    print('Maximum progesterone productivity on galactose:', round(progesterone_production,3), 'mmol/gDW*h') # calculate productivity
    print('Maximum progesterone yield on galactose:', round(max_yield_gal,3), 'mmol progesterone/mmol galactose') # calculate yield on galactose

Medium with glucose (before): {'r_1654': 1000.0, 'r_1714': 1.0, 'r_1832': 1000.0, 'r_1861': 1000.0, 'r_1992': 1000.0, 'r_2005': 1000.0, 'r_2020': 1000.0, 'r_2049': 1000.0, 'r_2060': 1000.0, 'r_2100': 1000.0, 'r_4593': 1000.0, 'r_4594': 1000.0, 'r_4595': 1000.0, 'r_4596': 1000.0, 'r_4597': 1000.0, 'r_4600': 1000.0}

Medium with galactose (after): {'r_1654': 1000.0, 'r_1718': 1.0, 'r_1832': 1000.0, 'r_1861': 1000.0, 'r_1992': 1000.0, 'r_2005': 1000.0, 'r_2020': 1000.0, 'r_2049': 1000.0, 'r_2060': 1000.0, 'r_2100': 1000.0, 'r_4593': 1000.0, 'r_4594': 1000.0, 'r_4595': 1000.0, 'r_4596': 1000.0, 'r_4597': 1000.0, 'r_4600': 1000.0}

When objective is set to only progesterone production:
Galactose uptake: 1.0 mmol/gDW*h
Maximum growth: 0.03 /h
Maximum progesterone productivity on galactose: 1.0 mmol/gDW*h
Maximum progesterone yield on galactose: 1.0 mmol progesterone/mmol galactose

When objective is set at both growth and progesterone production:
Medium before: {'r_1654': 1000.0, 'r_1714': 1